In [87]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pickle



In [ ]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

m = A.shape[0]

np.random.seed(0)
N = np.random.randn(A.shape[0], A.shape[1])
G = np.random.randn(A.shape[0], A.shape[1])

learning_rate = 0.01
iterations = 10000



N = np.random.randn(m, m)
G = np.random.randn(m, m)

learning_rate = 0.001

U, S, Vt = np.linalg.svd(A, full_matrices=False)


for _ in range(iterations):
    Z = N @ A  # Z = NT * A
    Y = G @ Z  # Y = G * Z
    
    dG = (Y - A) @ Z.T
    dN = G.T @ (Y - A) @ A.T

    G -= learning_rate * dG
    N -= learning_rate * dN

U_approx = G

frobenius_norm = np.linalg.norm(U - U_approx, 'fro')

U, U_approx, frobenius_norm

In [ ]:
import numpy as np

def initialize_weights(rows, cols):
    return np.random.randn(rows, cols) * 0.01, np.random.randn(cols, rows) * 0.01

def dgha_update_adam(G, N, P, learning_rate=0.001, beta1=0.5, beta2=0.9, epsilon=1e-8):
    U = np.dot(G, P)
    V = np.dot(N.T, U)
    
    m_G = np.zeros_like(G)
    v_G = np.zeros_like(G)
    m_N = np.zeros_like(N)
    v_N = np.zeros_like(N)
    
    G_delta = np.zeros_like(G)
    N_delta = np.zeros_like(N)
    
    for i in range(U.shape[1]):
        grad_G = np.outer(V[:, i], U[:, i]) - np.dot(np.outer(V[:, i], V[:, i].T), G)
        grad_N = np.outer(V[:, i], P[:, i]) - np.dot(np.outer(V[:, i], V[:, i].T), N.T)
        
        m_G = beta1 * m_G + (1 - beta1) * grad_G
        v_G = beta2 * v_G + (1 - beta2) * (grad_G ** 2)
        m_N = beta1 * m_N + (1 - beta1) * grad_N
        v_N = beta2 * v_N + (1 - beta2) * (grad_N ** 2)
        
        m_G_hat = m_G / (1 - beta1 ** (i + 1))
        v_G_hat = v_G / (1 - beta2 ** (i + 1))
        m_N_hat = m_N / (1 - beta1 ** (i + 1))
        v_N_hat = v_N / (1 - beta2 ** (i + 1))
        
        G_delta += learning_rate * m_G_hat / (np.sqrt(v_G_hat) + epsilon)
        N_delta += learning_rate * m_N_hat / (np.sqrt(v_N_hat) + epsilon)
    
    G += G_delta
    N += N_delta
    
    return G, N

P = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float)
P = P / np.linalg.norm(P)  # Нормализация P
G, N = initialize_weights(3, 3)

for _ in range(500000):
    G, N = dgha_update_adam(G, N, P, learning_rate=1e-8, beta1=0.99, beta2=0.99)


print("G:\n", G)
print("N:\n", N)
print("Approximate P^-1 or I:\n", np.dot(N, G))
bound = "==="
print(f" {bound * 30} RESULTS {bound * 30}")


U_svd, _, _ = np.linalg.svd(P)

U_alg = np.dot(N, G)

difference_norm_U = np.linalg.norm(U_alg - U_svd)

print("Difference norm between U from algorithm and U from SVD:", difference_norm_U)


In [ ]:
U, S, VT = np.linalg.svd(P)
P_reconstructed = np.dot(U, np.dot(np.diag(S), VT))
NG = np.dot(N, G)
print("NG reconstructed:\n", NG)
print("Original P reconstructed from SVD:\n", P_reconstructed)

difference_norm = np.linalg.norm(P_reconstructed - NG)
print("Difference norm:", difference_norm)


In [179]:
def scaledata(datain, minval, maxval):
    datain = datain.astype(float)  # Явное приведение к типу float
    dataout = datain - np.min(datain)
    dataout = (dataout / np.ptp(dataout)) * (maxval - minval)
    dataout = dataout + minval
    return dataout

In [175]:
def rgb2gray(rgb):
    datain = datain.astype(float)
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [161]:
import pickle

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [163]:
path = '/Users/turbomuza/Documents/diplom/'

In [ ]:
meta_file = path + 'batches.meta'
metadata = unpickle(meta_file)

for x, y in metadata.items():
  print(f"{x}:{y}")

In [ ]:
data_file = path + 'data_batch_1'
datafile = unpickle(data_file)

print([i for i in datafile.keys()])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

images = datafile.get('data')
images_transposed = np.transpose(images, (0, 2, 1, 3))  # Поменять местами 1-ю и 2-ю оси

check_elem = images_transposed[0]
print(check_elem)

print(f'check_elem = {check_elem.shape}')

plt.imshow(check_elem)
plt.axis('off')
plt.show()
print(len(images_transposed))

In [ ]:
label_name = datafile.get('labels')
labels = np.array([0 for i in range(len(label_name))])

label_names = metadata['label_names']
print(label_names)

# print(label_name)
# print(labels)


In [168]:
def short_label(lbl, n):
    result = lbl
    if len(lbl) > n:
        result = lbl[:n]+'.'
    return result


def pretty_axis(plt, ax):
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_axis_off()
    plt.tick_params(
        left=False,
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False
    ) # labels along the bottom edge are off

In [ ]:
count_of_class = 1

print(count_of_class)

In [ ]:
fig = plt.figure(figsize=(count_of_class, 1))
gs = gridspec.GridSpec(4,count_of_class, figure=fig)

for i in range(count_of_class):
    idx = np.random.choice(np.where(labels == i)[0],1)[0]
    image = images[idx]
    ax0 = fig.add_subplot(gs[2, i])
    ax0.imshow(image)
    pretty_axis(plt,ax0)

    ax1 = fig.add_subplot(gs[0, i])
    ax1.text(0.5, 0.5, labels[idx], fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax1)

    name = short_label(label_names[labels[idx]],6)
    ax2 = fig.add_subplot(gs[1, i])
    ax2.text(0.5, 0.5, name, fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax2)

    ax3 = fig.add_subplot(gs[3, i])
    ax3.text(0.5, 0.5, sum(labels == i), fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax3)

In [171]:
def short_label(lbl, n):
    result = lbl
    if len(lbl) > n:
        result = lbl[:n]+'.'
    return result

In [ ]:
min_images_in_class = 1e7
for i in range(count_of_class):
    size_of_class = sum(labels == i)
    if (min_images_in_class > size_of_class):
        min_images_in_class = size_of_class

print(min_images_in_class)
min_images_in_class = 5

In [ ]:
test_image_index = 1
test_image = images[test_image_index]
fig = plt.figure(figsize=(6, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_title('colored\n' + label_names[labels[test_image_index]])
plt.imshow(test_image)

ax1.set_yticklabels([])
ax1.set_xticklabels([])

gray_test_image = rgb2gray(test_image)
ax2 = fig.add_subplot(1, 2, 2)
plt.imshow(gray_test_image, cmap='gray')
ax2.set_title('gray\n' + label_names[labels[test_image_index]])
ax2.set_yticklabels([])
ax2.set_xticklabels([])

plt.tight_layout()
plt.show()

In [ ]:
len(labels == 0)

In [151]:
bottle_nack = 0.3
p = 100

In [ ]:
D = np.empty((count_of_class, min_images_in_class), dtype=object)

for i in range(count_of_class):
    idxs = np.where(labels == i)[0]
    for j, image_index in enumerate(idxs):
        if (j < min_images_in_class):
            print("===")
            print(scaledata(rgb2gray(images[image_index]),0,1))
            D[i, j] = scaledata(rgb2gray(images[image_index]),0,1)
        else:
            break

print(D)

In [ ]:
D = np.empty((count_of_class, min_images_in_class), dtype=object)

idxs = np.where(labels == 0)[0]
print(idxs)
for j, image_index in enumerate(idxs):
    if image_index > 4:
            # print(f"counter == {counter}")
        break
    if (j <= 4):
        D[i, j] = scaledata(rgb2gray(images_transposed[image_index]), 0, 1)
        print("====")
    else:
        break

print(D)

wdth = D[0, 0].shape[0]
hdth = D[0, 0].shape[1]
n = wdth * hdth
N = count_of_class * min_images_in_class


X = np.zeros((n, N))
k = 0
for i in range(count_of_class):
    for j in range(min_images_in_class):
        X[:, k] = np.reshape(D[i, j], (wdth*hdth, 1)).T
        k += 1

ones_vector = np.ones((X.shape[1], 1))
Xt = X - X.dot(ones_vector).dot(ones_vector.T) / N
print(Xt)

In [ ]:
UU = U_approx
T = np.eye(UU.shape[1]) * 0.001



print(f"U == {U.shape}")
print(f"S == {S.shape}")
print(f"V == {V.shape}")
print(f"UU == {UU.shape}")
print(f'T.shape == {T.shape}')

def activation(u):
    coeff_a = 0
    dx = 0.0000001
    # coeff_k1 = 10/dx #0.5
    coeff_k1 = 0.5
    retval = 1 / (1 + np.exp(-2.0 * coeff_k1 * (u + coeff_a)))
    return retval

coeff_a = 0;
dx = 0.0000001
coeff_k1 = 10/dx

alpha0 = 1 / (np.exp(-2*coeff_a*coeff_k1) + 1);
alpha1 = 2 * coeff_k1 * np.exp(2*coeff_a*coeff_k1) / ( np.exp(2*coeff_a*coeff_k1) + 1 )**2;

print(np.ones((p, 1)).shape)
print(np.linalg.inv(T).shape)

W1 = (1/alpha1) * T @ UU.T
print(f'W1.shape == {W1.shape}')
W2 = UU @ np.linalg.inv(T)
print(f'W2.shape == {W2.shape}')

w1 = -(1/alpha1) * T @ UU.T @ (X @ np.ones((N, 1)) / N)
print(f'w1.shape == {w1.shape}')
# w2 = (np.eye(n) - UU @ UU.T) @ (X @ np.ones((N, 1)) / N) - alpha0 * UU @ np.linalg.inv(T) @ np.ones((p, 1)) - alpha1 * UU @ np.linalg.inv(T) @ w1
temp_w2 = (np.eye(UU.shape[0]) - UU @ UU.T) @ (X @ np.ones((N, 1)) / N)
w2 = temp_w2.T - alpha0 * (UU @ np.linalg.inv(T) @ np.ones((UU.shape[1], 1))) - alpha1 * (UU @ np.linalg.inv(T) @ w1)

In [ ]:
XX = np.empty((hdth, 0))

YY = np.empty((hdth, 0))

fig, ((ax1), (ax2)) = plt.subplots(2, 1,figsize=(10, 2), dpi=180)

for digit in range(1):
    image_num = np.random.randint(min_images_in_class)
    B = np.dot(W1, D[digit, image_num].reshape(n, 1)) + w1
    H = activation(B)
    Y = np.dot(W2, H) + w2
    XX = np.hstack((XX, D[digit, image_num].reshape(wdth, hdth)))
    YY = np.reshape(Y, (hdth, wdth, -1))

ax1.imshow(XX, cmap='gray', aspect='equal')
ax1.axis('off')
ax2.imshow(YY[:, :, 0], cmap='gray', aspect='equal')
ax2.axis('off')

In [16]:
D = np.empty((count_of_class, min_images_in_class), dtype=object)

for i in range(count_of_class):
    idxs = np.where(labels == i)[0]
    for j, image_index in enumerate(idxs):
        if (j < min_images_in_class):
            D[i, j] = scaledata(rgb2gray(images[image_index]),0,1)
        else:
            break;

In [ ]:
import numpy as np
from sklearn.decomposition import IncrementalPCA

count_of_class = 1
min_images_in_class = 1078
batch_size = 100
p = 100
wdth = D[0, 0].shape[0]
hdth = D[0, 0].shape[1]
n = wdth * hdth  # wdth и hdth должны быть заданы ранее
N = count_of_class * min_images_in_class

# Инициализация Incremental PCA
ipca = IncrementalPCA(n_components=p, batch_size=batch_size)

# Подготовка данных
X = np.zeros((n, batch_size))
batch_index = 0
T = np.eye(p) * 0.001
inv_T = np.linalg.inv(T)
alpha1 = 0.001 
alpha0 = 0.0001

w1_accumulator = np.zeros((p, 1))
w2_accumulator = np.zeros((wdth * hdth, 1))
batch_count = 0

k = 0 

for i in range(count_of_class):
    for j in range(min_images_in_class):
        X[:, batch_index] = np.reshape(D[i, j], (wdth*hdth, 1)).T
        batch_index += 1
        k += 1
        if batch_index == batch_size or k == N:
            mean_X = X @ np.ones((batch_index, 1)) / batch_index
            Xt = X - mean_X
            ipca.partial_fit(Xt.T)

            U_approx = ipca.components_.T
            w1_batch = -(1 / alpha1) * T @ U_approx.T @ mean_X
            temp_w2 = (np.eye(U_approx.shape[0]) - U_approx @ U_approx.T) @ mean_X
            w2_batch = temp_w2.T - alpha0 * (U_approx @ inv_T @ np.ones((p, 1))) - alpha1 * (U_approx @ inv_T @ w1_batch)

            w1_accumulator += w1_batch
            w2_accumulator += w2_batch
            batch_count += 1

            X = np.zeros((n, batch_size))
            batch_index = 0

w1 = w1_accumulator / batch_count
w2 = w2_accumulator / batch_count

print(f'Final w1 shape: {w1.shape}, Final w2 shape: {w2.shape}')




In [ ]:
w1_accumulator = np.zeros((p, 1))
w2_accumulator = np.zeros((wdth * hdth, 1))

for i in range(count_of_class):
    for j in range(min_images_in_class):
        X[:, batch_index] = np.reshape(D[i, j], (wdth*hdth, 1)).T
        batch_index += 1
        k += 1
        
        if batch_index == batch_size or k == N:
            mean_X = X @ np.ones((batch_index, 1)) / batch_index
            Xt = X - mean_X
            ipca.partial_fit(Xt.T)
            w1_batch = -(1 / alpha1) * T @ U_approx.T @ mean_X
            temp_w2 = (np.eye(U_approx.shape[0]) - U_approx @ U_approx.T) @ mean_X
            w2_batch = temp_w2.T - alpha0 * (U_approx @ inv_T @ np.ones((p, 1))) - alpha1 * (U_approx @ inv_T @ w1_batch)

            w1_accumulator += w1_batch
            w2_accumulator += w2_batch
            batch_count += 1

            X = np.zeros((n, batch_size))
            batch_index = 0

In [19]:
bottle_nack = 0.3
p = 100

In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
from scipy.linalg import inv, solve

U, S, V = svds(Xt, k=p)

UU = U
# T = np.random.rand(p, p) * 0.01
T = np.eye(p) * 0.001
try:
    inv_T = inv(T)
except np.linalg.LinAlgError:
    print("lala")
    inv_T = solve(T, np.eye(T.shape[0]))


In [24]:
def activation(u):
    coeff_a = 0
    dx = 0.0000001
    # coeff_k1 = 10/dx #0.5
    coeff_k1 = 0.5
    retval = 1 / (1 + np.exp(-2.0 * coeff_k1 * (u + coeff_a)))
    return retval

coeff_a = 0;
dx = 0.0000001
coeff_k1 = 10/dx

alpha0 = 1 / (np.exp(-2*coeff_a*coeff_k1) + 1);
alpha1 = 2 * coeff_k1 * np.exp(2*coeff_a*coeff_k1) / ( np.exp(2*coeff_a*coeff_k1) + 1 )**2;

In [ ]:
print(np.ones((p, 1)).shape)
print(np.linalg.inv(T).shape)

In [ ]:
W1 = (1/alpha1) * T @ UU.T
print(f'W1.shape == {W1.shape}')
W2 = U @ np.linalg.inv(T)
print(f'W2.shape == {W2.shape}')

w1 = -(1/alpha1) * T @ U.T @ (X @ np.ones((N, 1)) / N)
print(f'w1.shape == {w1.shape}')
# w2 = (np.eye(n) - UU @ UU.T) @ (X @ np.ones((N, 1)) / N) - alpha0 * UU @ np.linalg.inv(T) @ np.ones((p, 1)) - alpha1 * UU @ np.linalg.inv(T) @ w1
temp_w2 = (np.eye(U.shape[0]) - U @ U.T) @ (X @ np.ones((N, 1)) / N)
w2 = temp_w2.T - alpha0 * (U @ np.linalg.inv(T) @ np.ones((p, 1))) - alpha1 * (U @ np.linalg.inv(T) @ w1)

In [ ]:
XX = np.empty((hdth, 0))

YY = np.empty((hdth, 0))

fig, ((ax1), (ax2)) = plt.subplots(2, 1,figsize=(10, 2), dpi=180)

for digit in range(1):
    image_num = np.random.randint(min_images_in_class)
    B = np.dot(W1, D[digit, image_num].reshape(n, 1)) + w1
    H = activation(B)
    Y = np.dot(W2, H) + w2
    XX = np.hstack((XX, D[digit, image_num].reshape(wdth, hdth)))
    YY = np.reshape(Y, (hdth, wdth, -1))

ax1.imshow(XX, cmap='gray', aspect='equal')
ax1.axis('off')
ax2.imshow(YY[:, :, 0], cmap='gray', aspect='equal')
ax2.axis('off')
